In [20]:
# Importing libraries
import os
import warnings
warnings.filterwarnings('ignore')

from datetime import datetime, timedelta
import math
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import scipy
import yfinance as yf

from neurotrader.download_data import download_df
from neurotrader.directional_change import directional_change
from neurotrader.perceptually_important import find_pips

# Reversal Pattern Detection Algorithm
- Look at the 1 day intervals for now
- Want to look at min by min at some point
- Goal: If a pattern is detected, how much wieght should we give the buying/selling power of that pattern
- AKA HOW ACCURATE THAT PATTERN IS FOR a certain stock
- Look for a way to add an "Early Detection" of a pattern. Do this by checking for ratios not including the point "D"

## TODO:
- Research and add different pattern ratios
- GARTLEY, BAT, BUTTERFLY, CRAB, DEEP_CRAB, CYPHER, SHARK
- Run each program to see output, see how to generate data we can add to neural network
- Directional change can be innacurate on the beginning of the data
- 
 

In [1]:
# Trendline parameter
lookback = 30

data = download_df('AAPL', '1y')
data.head(1)

NameError: name 'download_df' is not defined

# Testing Different Algorithms for finding local extrema
- Rolling Window
- Directional Change
- Perceptually Important Points

In [18]:
# ROLLING WINDOW ALGORITHM
# Use Scipy to get local extrema. BE CAREFUL NOT TO CHEAT WITH FUTURE DATA
# Use close data because it is more stable than adj high and adj low
arr = data['close'].to_numpy()
bottoms = scipy.signal.argrelextrema(arr, np.less, order=3)
tops = scipy.signal.argrelextrema(arr, np.greater, order=3)
print(bottoms[0][0:11])
print(tops[0][0:11])

[  2  12  18  31  41  50  63  71  90 108 118]
[ 7 14 19 23 29 38 47 57 69 77 87]


In [ ]:
# Directional Change Algorithm

In [25]:
# Perceptually Important Points
PIP = 10
pips_x, pips_y = find_pips(arr, PIP, 1)
print(pips_x)
print(pips_y)

[0, 53, 57, 102, 118, 129, 139, 156, 198, 252]
[162.02999877929688, 185.27000427246094, 193.97000122070312, 189.6999969482422, 170.42999267578125, 180.7100067138672, 166.88999938964844, 191.4499969482422, 195.17999267578125, 167.62649536132812]
